# PySpark - Работа с RDD (Resilient Distributed Datasets)

## Введение в RDD
RDD (Resilient Distributed Datasets) - это основная абстракция в Apache Spark. RDD представляет собой неизменяемую распределенную коллекцию объектов, которая может обрабатываться параллельно. В этом ноутбуке мы рассмотрим основные концепции и операции с RDD в PySpark.

## Содержание
1. Инициализация Spark и создание SparkContext
2. Создание RDD
3. Базовые операции с RDD
4. Трансформации RDD
5. Actions (действия) над RDD
6. Парные RDD и операции с ключами
7. Сохранение и загрузка RDD
8. Практические примеры

## 1. Инициализация Spark и создание SparkContext

In [ ]:
!pip install findspark numpy matplotlib pandas

In [ ]:
import findspark
findspark.init()

In [ ]:
# Импорт необходимых библиотек

from pyspark import SparkConf, SparkContext
import os
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Создание SparkConf и SparkContext
conf = SparkConf().setAppName("RDD Practice 2").setMaster("local[*]")
sc = SparkContext(conf=conf)

# Проверка версии Spark
print(f"Spark Version: {sc.version}")

## 2. Создание RDD
Существует несколько способов создания RDD в Spark:

### Создание RDD из коллекции Python

In [ ]:
# Создание RDD из Python-списка
data = [1, 2, 3, 4, 5]
rdd = sc.parallelize(data)
print(f"RDD: {rdd}")
print(f"RDD Content: {rdd.collect()}")

### Создание RDD из внешних файлов

In [ ]:
# Создадим текстовый файл для примера
!echo "Hello Spark" > example.txt
!echo "RDD is fundamental abstraction in Spark" >> example.txt
!echo "It represents a resilient distributed dataset" >> example.txt
# если выполняем на кластере, то используем hdfs
!hdfs dfs -ls /user
!hdfs dfs -mkdir -p /user/ubuntu
!hadoop fs -put -f example.txt /user/ubuntu/example.txt

In [ ]:
!hdfs dfs -ls /user/ubuntu

In [ ]:
# Создание RDD из текстового файла
text_rdd = sc.textFile("example.txt")
print(f"Text RDD Content: {text_rdd.collect()}")

### Создание RDD с определенным числом партиций

In [ ]:
# Создание RDD с 4 партициями
data_partitioned = sc.parallelize(range(10), 4)
print(f"Number of partitions: {data_partitioned.getNumPartitions()}")
print(f"Partitioned RDD content: {data_partitioned.collect()}")
print(f"Partitioned RDD glom: {data_partitioned.glom().collect()}")

## 3. Базовые операции с RDD

### Информация об RDD

In [ ]:
# Создание RDD из Python-списка
data = [1, 2, 3, 4, 5]
rdd = sc.parallelize(data)
print(f"RDD: {rdd}")
print(f"RDD Content: {rdd.collect()}")

In [ ]:
# Получение информации о партициях
num_partitions = rdd.getNumPartitions()
print(f"Number of partitions: {num_partitions}")

# Оценка размера RDD
print(f"RDD count: {rdd.count()}")

# Просмотр содержимого партиций с помощью glom()
print(f"Content of each partition: {rdd.glom().collect()}")

## 4. Трансформации RDD
Трансформации создают новый RDD из существующего. Они являются "ленивыми", то есть вычисляются только при вызове действия.

### map() - применение функции к каждому элементу


In [ ]:
# Применение функции map для возведения в квадрат
squared_rdd = rdd.map(lambda x: x**2)
print(f"Original RDD: {rdd.collect()}")
print(f"Squared RDD: {squared_rdd.collect()}")

### filter() - фильтрация элементов по условию

In [ ]:
# Фильтрация четных чисел
even_rdd = rdd.filter(lambda x: x % 2 == 0)
print(f"Original RDD: {rdd.collect()}")
print(f"Even numbers only: {even_rdd.collect()}")

### flatMap() - генерация 0 или более выходных элементов из каждого входного

In [ ]:
# Разделение строк на слова с помощью flatMap
words_rdd = text_rdd.flatMap(lambda line: line.split())
print(f"Original text RDD: {text_rdd.collect()}")
print(f"Words RDD (after flatMap): {words_rdd.collect()}")

### distinct() - удаление дубликатов

In [ ]:
# Создание RDD с дубликатами
dup_data = [1, 2, 2, 3, 3, 3, 4, 5, 5]
dup_rdd = sc.parallelize(dup_data)

In [ ]:
# Получение уникальных элементов
distinct_rdd = dup_rdd.distinct()
print(f"RDD with duplicates: {dup_rdd.collect()}")
print(f"RDD after distinct(): {distinct_rdd.collect()}")

### sample() - выборка элементов

In [ ]:
# Выборка с заменой
sample_with_replacement = rdd.sample(True, 0.5, seed=42)
print(f"Sample with replacement: {sample_with_replacement.collect()}")

In [ ]:
# Выборка без замены
sample_without_replacement = rdd.sample(False, 0.5, seed=42)
print(f"Sample without replacement: {sample_without_replacement.collect()}")

### union(), intersection(), subtract() - теоретико-множественные операции

In [ ]:
# Создание второго RDD
rdd2 = sc.parallelize([3, 4, 5, 6, 7])

In [ ]:
# Объединение RDD
union_rdd = rdd.union(rdd2)
print(f"RDD1: {rdd.collect()}")
print(f"RDD2: {rdd2.collect()}")
print(f"Union: {union_rdd.collect()}")


In [ ]:
# Пересечение RDD
intersection_rdd = rdd.intersection(rdd2)
print(f"Intersection: {intersection_rdd.collect()}")


In [ ]:
# Разность RDD
subtract_rdd = rdd.subtract(rdd2)
print(f"RDD1 - RDD2: {subtract_rdd.collect()}")



## 5. Actions (действия) над RDD
Действия возвращают значения из RDD в программу драйвера или записывают их во внешнюю систему хранения. Они запускают вычисления.

### collect() - получение всех элементов



In [ ]:
# Сбор всех элементов RDD в драйвер
collected_data = rdd.collect()
print(f"Collected data: {collected_data}")

### count() - подсчет элементов


In [ ]:
# Подсчет количества элементов
count = rdd.count()
print(f"Count: {count}")

### first() - получение первого элемента


In [ ]:
# Получение первого элемента
first_element = rdd.first()
print(f"First element: {first_element}")


### take(n) - получение n элементов

In [ ]:
# Получение первых n элементов
first_3 = rdd.take(3)
print(f"First 3 elements: {first_3}")


### reduce() - агрегирование элементов

In [31]:
# Сумма всех элементов с помощью reduce
sum_of_elements = rdd.reduce(lambda a, b: a + b)
print(f"Sum of all elements: {sum_of_elements}")

# Произведение всех элементов
product_of_elements = rdd.reduce(lambda a, b: a * b)
print(f"Product of all elements: {product_of_elements}")

Sum of all elements: 15
Product of all elements: 120


### foreach() - выполнение действия для каждого элемента

In [32]:
# Вывод каждого элемента (в логи исполнителей, не в драйвер)
rdd.foreach(lambda x: print(f"Element: {x}"))

## 6. Парные RDD и операции с ключами
Парные RDD - это RDD, элементами которого являются пары (ключ, значение). Они поддерживают особые операции, такие как groupByKey, reduceByKey и др.

### Создание парного RDD



In [33]:
# Создание парного RDD
pairs = [("a", 1), ("b", 2), ("a", 3), ("c", 4), ("b", 5), ("c", 6)]
pairs_rdd = sc.parallelize(pairs)
print(f"Pairs RDD: {pairs_rdd.collect()}")

Pairs RDD: [('a', 1), ('b', 2), ('a', 3), ('c', 4), ('b', 5), ('c', 6)]


### reduceByKey() - агрегирование значений по ключу

In [34]:
# Суммирование значений по ключу
sums_by_key = pairs_rdd.reduceByKey(lambda a, b: a + b)
print(f"Sum by key: {sums_by_key.collect()}")

Sum by key: [('b', 7), ('c', 10), ('a', 4)]


### groupByKey() - группировка значений по ключу

In [35]:
# Группировка значений по ключу
grouped_by_key = pairs_rdd.groupByKey().mapValues(list)
print(f"Grouped by key: {grouped_by_key.collect()}")

Grouped by key: [('b', [2, 5]), ('c', [4, 6]), ('a', [1, 3])]


### sortByKey() - сортировка по ключу

In [36]:
# Сортировка по ключу
sorted_by_key = pairs_rdd.sortByKey()
print(f"Sorted by key: {sorted_by_key.collect()}")

# Сортировка по ключу в обратном порядке
sorted_by_key_desc = pairs_rdd.sortByKey(ascending=False)
print(f"Sorted by key (descending): {sorted_by_key_desc.collect()}")

Sorted by key: [('a', 1), ('a', 3), ('b', 2), ('b', 5), ('c', 4), ('c', 6)]
Sorted by key (descending): [('c', 4), ('c', 6), ('b', 2), ('b', 5), ('a', 1), ('a', 3)]


### join() - соединение парных RDD

In [37]:
# Создание второго парного RDD
pairs2 = [("a", "apple"), ("b", "banana"), ("c", "cherry")]
pairs_rdd2 = sc.parallelize(pairs2)

In [38]:
# Внутреннее соединение
joined_rdd = pairs_rdd.join(pairs_rdd2)
print(f"RDD1: {pairs_rdd.collect()}")
print(f"RDD2: {pairs_rdd2.collect()}")
print(f"Joined RDD: {joined_rdd.collect()}")

RDD1: [('a', 1), ('b', 2), ('a', 3), ('c', 4), ('b', 5), ('c', 6)]
RDD2: [('a', 'apple'), ('b', 'banana'), ('c', 'cherry')]
Joined RDD: [('a', (1, 'apple')), ('a', (3, 'apple')), ('b', (2, 'banana')), ('b', (5, 'banana')), ('c', (4, 'cherry')), ('c', (6, 'cherry'))]


In [39]:
# Левое внешнее соединение
left_joined = pairs_rdd.leftOuterJoin(pairs_rdd2)
print(f"Left joined: {left_joined.collect()}")

Left joined: [('a', (1, 'apple')), ('a', (3, 'apple')), ('b', (2, 'banana')), ('b', (5, 'banana')), ('c', (4, 'cherry')), ('c', (6, 'cherry'))]


In [40]:
# Правое внешнее соединение
right_joined = pairs_rdd.rightOuterJoin(pairs_rdd2)
print(f"Right joined: {right_joined.collect()}")

Right joined: [('a', (1, 'apple')), ('a', (3, 'apple')), ('b', (2, 'banana')), ('b', (5, 'banana')), ('c', (4, 'cherry')), ('c', (6, 'cherry'))]


## 7. Сохранение и загрузка RDD

### Сохранение в текстовый файл



In [51]:
# Сохранение RDD в текстовый файл
sums_by_key.saveAsTextFile("key_sums.txt")

Py4JJavaError: An error occurred while calling o718.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory hdfs://rc1a-dataproc-m-op86a0r1gkd8750i.mdb.yandexcloud.net/user/ubuntu/key_sums.txt already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:294)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1090)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:388)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1

In [52]:
!ls -la

total 52
drwxr-xr-x 7 ubuntu ubuntu 4096 Jun 10 18:22 .
drwxr-xr-x 4 root   root   4096 Jun 10 16:55 ..
-rw------- 1 ubuntu ubuntu    5 Jun 10 18:17 .bash_history
-rw-r--r-- 1 ubuntu ubuntu  220 Feb 25  2020 .bash_logout
-rw-r--r-- 1 ubuntu ubuntu 3933 Jun  4 19:32 .bashrc
drwx------ 5 ubuntu ubuntu 4096 Jun 10 18:21 .cache
drwxrwxr-x 3 ubuntu ubuntu 4096 Jun 10 18:21 .config
-rw-rw-r-- 1 ubuntu ubuntu   98 Jun 10 18:22 example.txt
drwxr-xr-x 5 ubuntu ubuntu 4096 Jun 10 18:20 .ipython
drwxrwxr-x 4 ubuntu ubuntu 4096 Jun 10 18:20 .local
-rw-r--r-- 1 ubuntu ubuntu  807 Feb 25  2020 .profile
drwx------ 2 ubuntu ubuntu 4096 Jun  4 19:22 .ssh
-rw-r--r-- 1 ubuntu ubuntu    0 Jun  4 19:22 .sudo_as_admin_successful
-rwxr-xr-x 1 ubuntu ubuntu 1416 Jun 10 17:02 upload_data_to_hdfs.sh


In [ ]:
# Просмотр сохраненных файлов
!ls -l key_sums

ls: cannot access '/user/ubuntu/key_sums': No such file or directory


In [43]:
# Просмотр содержимого сохраненных файлов
!cat key_sums/part-*

cat: 'key_sums/part-*': No such file or directory


In [ ]:
# Удаление созданных файлов
!rm -rf key_sums/
!rm example.txt

In [ ]:
!ls -la

total 48
drwxr-xr-x 7 ubuntu ubuntu 4096 Jun 10 18:34 .
drwxr-xr-x 4 root   root   4096 Jun 10 16:55 ..
-rw------- 1 ubuntu ubuntu    5 Jun 10 18:17 .bash_history
-rw-r--r-- 1 ubuntu ubuntu  220 Feb 25  2020 .bash_logout
-rw-r--r-- 1 ubuntu ubuntu 3933 Jun  4 19:32 .bashrc
drwx------ 5 ubuntu ubuntu 4096 Jun 10 18:21 .cache
drwxrwxr-x 3 ubuntu ubuntu 4096 Jun 10 18:21 .config
drwxr-xr-x 5 ubuntu ubuntu 4096 Jun 10 18:20 .ipython
drwxrwxr-x 4 ubuntu ubuntu 4096 Jun 10 18:20 .local
-rw-r--r-- 1 ubuntu ubuntu  807 Feb 25  2020 .profile
drwx------ 2 ubuntu ubuntu 4096 Jun  4 19:22 .ssh
-rw-r--r-- 1 ubuntu ubuntu    0 Jun  4 19:22 .sudo_as_admin_successful
-rwxr-xr-x 1 ubuntu ubuntu 1416 Jun 10 17:02 upload_data_to_hdfs.sh


## 8. Практические примеры

### Пример 1: Подсчет слов в тексте

In [ ]:
# Создадим текстовый файл с примером текста
text = """
Apache Spark is an open-source unified analytics engine for large-scale data processing.
Spark provides an interface for programming entire clusters with implicit data parallelism and fault tolerance.
Originally developed at the University of California, Berkeley's AMPLab, the Spark codebase was later donated to the Apache Software Foundation, which has maintained it since.
Spark provides an interface for programming entire clusters with implicit data parallelism and fault tolerance.
"""

with open("spark_text.txt", "w") as f:
    f.write(text)

In [ ]:
!hadoop fs -put -f spark_text.txt /user/ubuntu/spark_text.txt

In [ ]:
# Загрузка текста в RDD
text_rdd = sc.textFile("spark_text.txt")

# Подсчет слов (классическая задача WordCount)
word_counts = (text_rdd
    .flatMap(lambda line: line.split())
    .map(lambda word: (word.lower().strip(".,;:\"\'()[]{}"), 1))
    .reduceByKey(lambda a, b: a + b)
    .sortBy(lambda x: x[1], ascending=False)
)


print("Количество слов в тексте:")
for word, count in word_counts.collect():
    print(f"{word}: {count}")

# Визуализация топ-10 слов
top_10_words = word_counts.take(10)
words = [word for word, count in top_10_words]
counts = [count for word, count in top_10_words]

plt.figure(figsize=(12, 6))
plt.bar(words, counts)
plt.xlabel('Слова')
plt.ylabel('Количество')
plt.title('Топ-10 слов в тексте')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Пример 2: Оценка числа π методом Монте-Карло

In [ ]:
def inside_circle(p):
    """Проверяет, находится ли точка p внутри единичной окружности"""
    x, y = p
    return x*x + y*y < 1

# Число точек для оценки
num_samples = 1000000

# Генерация случайных точек
samples = sc.parallelize([
    (random.uniform(-1, 1), random.uniform(-1, 1)) 
    for _ in range(num_samples)
])

# Подсчет точек внутри окружности
count_inside = samples.filter(inside_circle).count()

# Оценка числа π
pi_estimate = 4.0 * count_inside / num_samples

print(f"Оценка числа π методом Монте-Карло: {pi_estimate}")
print(f"Истинное значение π: {np.pi}")
print(f"Погрешность: {abs(pi_estimate - np.pi)}")

# Визуализация метода Монте-Карло (на меньшем числе точек)
visualize_samples = 1000
points = [(random.uniform(-1, 1), random.uniform(-1, 1)) for _ in range(visualize_samples)]
inside = [p for p in points if inside_circle(p)]
outside = [p for p in points if not inside_circle(p)]

plt.figure(figsize=(8, 8))
plt.scatter([p[0] for p in inside], [p[1] for p in inside], color='blue', s=3, label='Inside')
plt.scatter([p[0] for p in outside], [p[1] for p in outside], color='red', s=3, label='Outside')
circle = plt.Circle((0, 0), 1, fill=False, color='black')
plt.gca().add_patch(circle)
plt.axis('equal')
plt.xlim(-1.05, 1.05)
plt.ylim(-1.05, 1.05)
plt.legend()
plt.title(f'Оценка числа π методом Монте-Карло: {4.0 * len(inside) / visualize_samples:.4f}')
plt.show()



In [ ]:
# Очистка после работы
!rm spark_text.txt

In [ ]:
# Остановка SparkContext
sc.stop()

print("RDD практика завершена!")